# Spatial Mapping

This notebook contains the code for and the final product of the active new building construction site scoping project&mdash;a geo-map of building lots still under construction in New York City.

## Introduction

### Classifying active construction lots

A lot is considered to be a building under construction, for the purposes of this project, if it has been issued a DOB construction permit which has not yet expired, but no certificate of occupancy is on record with the DOB with a date of granting that is more recent than the construction permit approval date.

Work can begin as soon as a construction permit is issued; permits usually last one year before they expire and require renewal. A building cannot actually be occupied until all inspections have taken place and a certificate of occupancy has been issued&mdash;thus the "C of O" approval is traditionally seen as the end of a construction project.

The single caveat is when a lot has recieved a construction permit but due to issues on the side of the permitee that lot never entered construction. Since permits expire after a year, unless a permitee is constantly renewing permits and not doing work I expect that this error is small.

It's also a bit of a philosophical question as to whether or not a lot which is effectively just an unused hole-in-the-ground is a construction site, an active construction site, or just a hole-in-the-ground.

Our classification is as tight as it can be, with respect to what the city regulates.

### Data processing

I use two data sources for this project. The first data source is the [DOB permit issuance dataset](https://data.cityofnewyork.us/Housing-Development/DOB-Permit-Issuance/ipu4-2q9a) on NYC Open Data, from which I retrieve a list of lots with non-expired construction permits (as of writing). The second data source is the DOB [BISweb interface](http://a810-bisweb.nyc.gov/bisweb/bsqpm01.jsp), which provides, in part, PDF copies of the certificates of occupancy that DOB has on its digital record (all recent ones; certificates going further back are more tenuous, with some document scans reaching back as far as ~1900). These are scraped using the `co_reader` module, a Python module that was written for the topically similar [NYC Construction Timeline project](https://github.com/ResidentMario/nyc-construction-timeline) which uses a text scanner to parse out dates from issued certificates.

This data processing is handled in the preceding `Active New Building Construction Site Data Join.ipynb` notebook.

### Dataset

At this point we have a file, `Active New Building Construction Sites.csv`, which contains a unified recordset of all of the active under-construction BINs in New York City (as well as some data about them, taken from the original permit dataset). This file is the source of the map generated by the notebook code in this file.

### Further work

The end product of this notebook is a mapping of active construction sites in New York City, as of the date of processing in mid-July 2016. However, this is functionally no more than a proof of concept. Constructing the dataset required reading in 4000 BISweb certificates, a herculean process that does not scale to operationalization as it requires probably over 24 hours overall to run. A finer-resolution active construction sites chart generated on a daily, monthly, or weekly basis would require an easier way to access this data (and one that wouldn't load down the DOB servers every time it is requested).

I suggest an Open Data dataset of certificate of occupancy releases.

### Small note

BIN `1012420`, address `507 WEST 28 STREET MANHATTAN`, has [65 replicitious certificates of occupancy](http://a810-bisweb.nyc.gov/bisweb/COsByLocationServlet?requestid=4&allbin=1012420), somehow. It correspond with a massive new apartment tower on the High Line ([source](https://newyorkyimby.com/category/507-west-28th-street), [source](http://www.6sqft.com/west-chelseas-tallest-tower-rises-and-finally-reveals-itself/), et. al.). Interesting shape issue.

## Map construction

In [4]:
import geopandas as gpd
import pandas as pd
import requests
import zipfile
import io
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option("max_columns", 500)

### Reading geospatial lot data

The MapPLUTO datasets are beefy ones, provided as seperate zipfiles for each of the boroughs. The following lines of code localize that data.

In [7]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/bk_mappluto_16v1.zip')
with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
#     ar.extract('BKMapPLUTO.shp')
#     ar.extract('BKMapPLUTO.shx')
    ar.extractall("data/brooklyn/")
del r

In [ ]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/bx_mappluto_16v1.zip')
with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
#     ar.extract('BXMapPLUTO.shp')
#     ar.extract('BXMapPLUTO.shx')
    ar.extractall("data/bronx/")
del r

In [ ]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/mn_mappluto_16v1.zip')
with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
#     ar.extract('MNMapPLUTO.shp')
#     ar.extract('MNMapPLUTO.shx')
    ar.extractall("data/manhattan/")
del r

In [ ]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/qn_mappluto_16v1.zip')
with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
#     ar.extract('QNMapPLUTO.shp')
#     ar.extract('QNMapPLUTO.shx')
    ar.extractall("data/queens/")
del r

In [ ]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/si_mappluto_16v1.zip')
with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
#     ar.extract('SIMapPLUTO.shp')
#     ar.extract('SIMapPLUTO.shx')
    ar.extractall("data/staten_island/")
del r

In [8]:
%ls

Active New Building Construction Site Data Join.ipynb
Active New Building Construction Site Spatial Map.ipynb
DOB Permit Issuance.csv
Latest Active New Building Permits.p
NYC Community Districts.geojson
Untitled.ipynb
active_sample_1.csv
co_reader.py
co_reader.pyc
data/
environment.yml
ghostdriver.log
temp.pdf


In [9]:
brooklyn = gpd.read_file("data/brooklyn/", layer='BKMapPLUTO')

KeyboardInterrupt: 

In [2]:
brooklyn.head(5)

### Reading city outline shapefile

Following Deena's lead on this, I am using the [NYC City Council GeoJson](https://github.com/dwillis/nyc-maps) from the public `nyc-maps` repository.

In [47]:
r = requests.get('https://raw.githubusercontent.com/dwillis/nyc-maps/master/city_council.geojson')
with open("NYC Community Districts.geojson", "w") as f:
    f.write(r.text)
del r

In [38]:
# gpd.read_file("BKMapPLUTO.shp")

KeyboardInterrupt: 